In [19]:
!pip install sklearn
!pip install torch
!pip install pandas
# !pip install PIL

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [20]:
!pip install tensorflow
!pip install proglearn

In [21]:
!git clone https://github.com/milesial/Pytorch-UNet

fatal: destination path 'Pytorch-UNet' already exists and is not an empty directory.


In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
dir_path = "/content/drive/MyDrive/ObjectDetection-Tasks"
filepath = os.path.join(dir_path, "Task-for-model-2")
dataset_path = os.path.join(filepath, 'retrainDataset/combinedRetrainData')  # New structured dataset location

labels_source = os.path.join(dataset_path, "labelData") # Change this directory target into the converted directory
images_source = os.path.join(dataset_path,  "imageData") # add this if in another directory , "captured_frames"

### Load data

In [24]:
import os
import numpy as np
from sklearn.model_selection import train_test_split

# Define your paths
image_dir = images_source
label_dir = labels_source

# Get all image files (assuming common image extensions)
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
image_files.sort()

# Get corresponding label files (adjust extension as needed)
label_files = [f.replace('.jpg', '.txt').replace('.png', '.txt') for f in image_files]

# Create full paths
image_paths = [os.path.join(image_dir, f) for f in image_files]
label_paths = [os.path.join(label_dir, f) for f in label_files]

# First split: separate train + val from test (80% train+val, 20% test)
train_val_images, test_images, train_val_labels, test_labels = train_test_split(
    image_paths, label_paths, test_size=0.2, random_state=42, shuffle=True
)

# Second split: separate train from val (75% train, 25% val of the 80%)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_val_images, train_val_labels, test_size=0.25, random_state=42, shuffle=True
)

print(f"Train image samples: {len(train_images)} ; Train label samples: {len(train_labels)}")
print(f"Validation image samples: {len(val_images)} ; Validation label samples: {len(val_labels)}")
print(f"Test image samples: {len(test_images)} ; Test label samples: {len(test_labels)}")

Train image samples: 195 ; Train label samples: 195
Validation image samples: 65 ; Validation label samples: 65
Test image samples: 65 ; Test label samples: 65


In [25]:
import json

# After creating splits, save them to files
splits = {
    'train': {
        'images': train_images,
        'labels': train_labels
    },
    'val': {
        'images': val_images,
        'labels': val_labels
    },
    'test': {
        'images': test_images,
        'labels': test_labels
    }
}

# Save to JSON
with open('/content/drive/MyDrive/ObjectDetection-Tasks/Task-for-model-2/retrainDataset/combinedRetrainData/dataset_splits.json', 'w') as f:
    # Convert Path objects to strings if necessary
    json_splits = {}
    for split_name, split_data in splits.items():
        json_splits[split_name] = {
            'images': [str(path) for path in split_data['images']],
            'labels': [str(path) for path in split_data['labels']]
        }
    json.dump(json_splits, f, indent=2)

In [26]:
import json
# Load splits later
def load_splits(split_file):
    with open(split_file, 'r') as f:
        splits = json.load(f)
    return splits

# Usage
splits = load_splits('/content/drive/MyDrive/ObjectDetection-Tasks/Task-for-model-2/retrainDataset/combinedRetrainData/dataset_splits.json')
train_images = splits['train']['images']
train_labels = splits['train']['labels']
test_images = splits['test']['images']
test_labels = splits['test']['labels']
val_images = splits['val']['images']
val_labels = splits['val']['labels']

### Train model

In [27]:
import tensorflow as tf
from tensorflow.keras import layers, Model, regularizers

def build_regularized_unet(input_shape=(256, 256, 3), num_classes=5):
    """
    Builds a simple U-Net inspired model with regularization for image segmentation.
    """
    # Input layer
    inputs = tf.keras.Input(shape=input_shape)

    # --- Data Augmentation (Applied as the first step) ---
    # Using a seed ensures the same transformation is applied to both image and mask :cite[1]
    x = layers.RandomFlip(mode="horizontal", seed=42)(inputs)
    x = layers.RandomRotation(factor=0.1, seed=42)(x)
    x = layers.RandomContrast(factor=0.1, seed=42)(x)

    # --- Encoder (Contracting Path) ---
    # Block 1
    c1 = layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4), padding='same')(x)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4), padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)
    p1 = layers.Dropout(0.2)(p1)  # Add dropout after pooling :cite[4]

    # Block 2
    c2 = layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4), padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(1e-4), padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)
    p2 = layers.Dropout(0.2)(p2)

    # --- Bottleneck ---
    bottleneck = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    bottleneck = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(bottleneck)

    # --- Decoder (Expansive Path) ---
    # Upsample and concatenate with skip connection from c2
    u1 = layers.UpSampling2D((2, 2))(bottleneck)
    u1 = layers.concatenate([u1, c2])
    u1 = layers.Dropout(0.2)(u1)
    u1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)
    u1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)

    # Final upsample and concatenate with skip connection from c1
    u2 = layers.UpSampling2D((2, 2))(u1)
    u2 = layers.concatenate([u2, c1])
    u2 = layers.Dropout(0.2)(u2)
    u2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u2)
    u2 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u2)

    # Output layer: Use a 1x1 convolution to map to the number of classes.
    # For binary segmentation, use 'sigmoid' activation; for multi-class, use 'softmax'.
    outputs = layers.Conv2D(num_classes, (1, 1), activation='softmax')(u2)

    model = Model(inputs, outputs)
    return model

# Build the model
model_unet = build_regularized_unet()
model_unet.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model_unet.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ random_flip_1       │ (None, 256, 256,  │          0 │ input_layer_1[0]… │
│ (RandomFlip)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ random_rotation_1   │ (None, 256, 256,  │          0 │ random_flip_1[0]… │
│ (RandomRotation)    │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ random_contrast_1   │ (None, 256, 256,  │          0 │ random_rotation_… │
│ (RandomContrast)    │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 256, 256,  │        896 │ random_contrast_… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 256, 256,  │      9,248 │ conv2d_11[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 128, 128,  │          0 │ conv2d_12[0][0]   │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 128, 128,  │          0 │ max_pooling2d_2[… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 128, 128,  │     18,496 │ dropout_4[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 128, 128,  │     36,928 │ conv2d_13[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 64, 64,    │          0 │ conv2d_14[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 64, 64,    │          0 │ max_pooling2d_3[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 64, 64,    │     73,856 │ dropout_5[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 64, 64,    │    147,584 │ conv2d_15[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_2     │ (None, 128, 128,  │          0 │ conv2d_16[0][0]   │
│ (UpSampling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 128, 128,  │          0 │ up_sampling2d_2[… │
│ (Concatenate)       │ 192)              │            │ conv2d_14[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 128, 128,  │          0 │ concatenate_2[0]

 Total params: 471,685 (1.80 MB)

 Trainable params: 471,685 (1.80 MB)

 Non-trainable params: 0 (0.00 B)

In [54]:
# Data Convert

import tensorflow as tf
from tensorflow import keras
import os
import numpy as np

# Define your image size and batch size
img_size = (256, 256)
batch_size = 32
num_classes = 5 # Define the number of classes to match the model

# 1. Prepare paths to your JPG images and segmentation masks
input_dir = "/content/drive/MyDrive/ObjectDetection-Tasks/Task-for-model-2/retrainDataset/combinedRetrainData/imageData/"
target_dir = "/content/drive/MyDrive/ObjectDetection-Tasks/Task-for-model-2/retrainDataset/combinedRetrainData/labelData/"

# Get sorted list of file paths (assuming image and label have same base name)
image_files = sorted([fname for fname in os.listdir(input_dir) if fname.lower().endswith((".jpg", ".jpeg", ".png"))])
label_files = sorted([fname for fname in os.listdir(target_dir) if fname.lower().endswith(".txt")])

# Ensure there's a corresponding label file for each image file
input_img_paths = []
target_img_paths = []

for img_file in image_files:
    base_name = os.path.splitext(img_file)[0]
    label_file = base_name + ".txt"
    if label_file in label_files:
        input_img_paths.append(os.path.join(input_dir, img_file))
        target_img_paths.append(os.path.join(target_dir, label_file))
    else:
        print(f"Warning: No corresponding label file found for image: {img_file}")


print(f"Number of samples found: {len(input_img_paths)}")

# 2. Create a TensorFlow Dataset
def get_dataset(batch_size, img_size, image_paths, label_paths):
    @tf.function # Use tf.function to compile the map function
    def load_img_masks(image_path, label_path):
        tf.print(f"Processing image: {image_path}") # Print current image path

        # Read the raw image file
        img_raw = tf.io.read_file(image_path)
        tf.print(f"Successfully read raw image file: {image_path}")

        # Try decoding as JPEG
        try:
            tf.print(f"Attempting to decode as JPEG: {image_path}")
            img = tf.io.decode_jpeg(img_raw, channels=3)
            tf.print(f"Successfully decoded image as JPEG: {image_path}")

            img = tf.image.resize(img, img_size)
            img = tf.image.convert_image_dtype(img, "float32")
            tf.print(f"Successfully resized and converted image: {image_path}")


        except tf.errors.InvalidArgumentError as e:
            tf.print(f"Error decoding image {image_path} as JPEG: {e}")
            # If JPEG decoding fails, return dummy data
            return tf.zeros(list(img_size) + [3], dtype=tf.float32), tf.zeros(list(img_size) + [num_classes], dtype=tf.uint8)
        except Exception as e:
             tf.print(f"An unexpected error occurred during JPEG image decoding for {image_path}: {e}")
             # Return a dummy image and mask for other exceptions
             return tf.zeros(list(img_size) + [3], dtype=tf.float32), tf.zeros(list(img_size) + [num_classes], dtype=tf.uint8)


        # Read and process the YOLO label file into a mask
        # Assuming YOLO format: class_id center_x center_y width height (normalized)
        # Need to convert bounding boxes to a segmentation mask

        # Create a blank mask image using TensorFlow with num_classes channels
        mask = tf.zeros(list(img_size) + [num_classes], dtype=tf.uint8)

        # Read bounding boxes from the label file
        try:
            tf.print(f"Processing label file: {label_path}")
            # Read the content of the label file
            label_content = tf.io.read_file(label_path)
            label_lines = tf.strings.split(label_content, '\n')

            # Iterate over each line in the label file
            for line in tf.strings.split(label_lines, ' '):
                if tf.shape(line)[0] == 5: # Ensure the line has 5 elements (class_id, cx, cy, w, h)
                    class_id = tf.strings.to_number(line[0], out_type=tf.int32)
                    center_x = tf.strings.to_number(line[1], out_type=tf.float32)
                    center_y = tf.strings.to_number(line[2], out_type=tf.float32)
                    width = tf.strings.to_number(line[3], out_type=tf.float32)
                    height = tf.strings.to_number(line[4], out_type=tf.float32)

                    # Convert normalized coordinates to pixel coordinates
                    img_width, img_height = tf.cast(img_size[1], tf.float32), tf.cast(img_size[0], tf.float32)
                    x_center = center_x * img_width
                    y_center = center_y * img_height
                    box_width = width * img_width
                    box_height = height * img_height

                    x_min = tf.cast(x_center - box_width / 2, tf.int32)
                    y_min = tf.cast(y_center - box_height / 2, tf.int32)
                    x_max = tf.cast(x_center + box_width / 2, tf.int32)
                    y_max = tf.cast(y_center + box_height / 2, tf.int32)

                    # Ensure coordinates are within image bounds
                    x_min = tf.maximum(0, x_min)
                    y_min = tf.maximum(0, y_min)
                    x_max = tf.minimum(tf.cast(img_width, tf.int32), x_max)
                    y_max = tf.minimum(tf.cast(img_height, tf.int32), y_max)


                    # Draw the bounding box on the mask (fill the corresponding class channel)
                    # Create a rectangle mask for the current bounding box for the specific class channel
                    if x_max > x_min and y_max > y_min and class_id < num_classes:
                        # Create a mask for this specific bounding box
                        box_mask = tf.ones([y_max - y_min, x_max - x_min, 1], dtype=tf.uint8) * 1

                        # Pad this box mask back to the original image size
                        box_mask = tf.image.pad_to_bounding_box(
                            box_mask, y_min, x_min, img_size[0], img_size[1]
                        )

                        # Expand dimensions to match mask shape (height, width, 1)
                        box_mask = tf.expand_dims(box_mask, axis=-1)

                        # Scatter the box mask into the correct class channel of the main mask
                        # Create an update tensor with the box mask value for the specific class channel
                        updates = tf.ones([img_size[0], img_size[1]], dtype=tf.uint8) * 1 # Value to set in the channel
                        updates = updates * tf.squeeze(box_mask, axis=-1) # Apply the box mask shape

                        # Create indices to scatter the updates into the correct channel
                        # Indices shape: (height, width, 1) - for each pixel, the class channel index
                        indices = tf.stack([
                            tf.range(img_size[0])[:, None] * tf.ones(img_size[1], dtype=tf.int32),
                            tf.ones([img_size[0], 1], dtype=tf.int32) * tf.range(img_size[1]),
                            tf.ones(img_size, dtype=tf.int32) * class_id # The class channel index
                        ], axis=-1)


                        # Use tf.tensor_scatter_nd_add or tf.tensor_scatter_nd_update
                        # tf.tensor_scatter_nd_add is good for overlapping boxes, adds 1 where they overlap
                        # tf.tensor_scatter_nd_update will just take the last value written to a pixel
                        # Let's use add for now to indicate overlaps, or you could use update if classes are mutually exclusive
                        # Note: scatter_nd_add requires input and updates to have the same type
                        updates = tf.cast(updates, tf.int32)
                        mask = tf.cast(mask, tf.int32)
                        mask = tf.tensor_scatter_nd_add(mask, indices, updates)
                        mask = tf.cast(mask, tf.uint8) # Convert back to uint8


                    else:
                        tf.print(f"Warning: Invalid bounding box dimensions or class_id for {label_path}")


        except Exception as e:
            tf.print(f"Error processing label file {label_path}: {e}")
            # Return an empty mask if there's an error
            mask = tf.zeros(list(img_size) + [num_classes], dtype=tf.uint8)


        # Resize to ensure correct size and dtype
        mask = tf.image.resize(mask, img_size, method="nearest")
        mask = tf.image.convert_image_dtype(mask, "uint8")


        return img, mask

    dataset = tf.data.Dataset.from_tensor_slices((input_img_paths, target_img_paths))
    dataset = dataset.map(load_img_masks, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size)
    return dataset

# Determine the split points based on the number of samples found
total_samples = len(input_img_paths)
train_size = int(0.8 * total_samples)
val_size = int(0.1 * total_samples)
test_size = total_samples - train_size - val_size # Remaining for test

# Create the training, validation, and test datasets
train_dataset = get_dataset(batch_size, img_size, input_img_paths[:train_size], target_img_paths[:train_size])
val_dataset = get_dataset(batch_size, img_size, input_img_paths[train_size:train_size + val_size], target_img_paths[train_size:train_size + val_size])
test_dataset = get_dataset(batch_size, img_size, input_img_paths[train_size + val_size:], target_img_paths[train_size + val_size:])

# Print dataset sizes (adjusting for potential partial batches)
def print_dataset_size(dataset, name):
    cardinality = tf.data.experimental.cardinality(dataset)
    if cardinality == tf.data.experimental.INFINITE_CARDINALITY:
        print(f"{name} dataset size: Infinite")
    elif cardinality == tf.data.experimental.UNKNOWN_CARDINALITY:
        print(f"{name} dataset size: Unknown")
    else:
        print(f"{name} dataset size: {cardinality.numpy() * batch_size}")

print_dataset_size(train_dataset, "Train")
print_dataset_size(val_dataset, "Validation")
print_dataset_size(test_dataset, "Test")

Number of samples found: 321
Train dataset size: 352
Validation dataset size: 352
Test dataset size: 352


In [50]:
import tensorflow as tf
import os

# Get the first image path from the list
if input_img_paths:
    sample_image_path = input_img_paths[0]
    print(f"Attempting to load and decode sample image: {sample_image_path}")

    try:
        # Read the raw image file
        img_raw = tf.io.read_file(sample_image_path)
        print(f"Successfully read raw image file: {sample_image_path}")

        # Attempt to decode the image
        try:
            # Try decoding as JPEG first, then PNG
            if tf.strings.regex_full_match(tf.strings.lower(sample_image_path), ".*\.jpg$|.*\.jpeg$"):
                print(f"Attempting to decode as JPEG: {sample_image_path}")
                img = tf.io.decode_jpeg(img_raw, channels=3)
            elif tf.strings.regex_full_match(tf.strings.lower(sample_image_path), ".*\.png$"):
                print(f"Attempting to decode as PNG: {sample_image_path}")
                img = tf.io.decode_png(img_raw, channels=3)
            else:
                print(f"Attempting to decode with decode_image: {sample_image_path}")
                img = tf.image.decode_image(img_raw, channels=3)

            print(f"Successfully decoded sample image with shape: {img.shape}")

            # Optional: Display the decoded image to verify
            # import matplotlib.pyplot as plt
            # plt.imshow(img.numpy())
            # plt.title("Decoded Sample Image")
            # plt.axis('off')
            # plt.show()

        except tf.errors.InvalidArgumentError as e:
            print(f"Error decoding sample image {sample_image_path}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred during sample image decoding for {sample_image_path}: {e}")

    except Exception as e:
        print(f"Error reading sample image file {sample_image_path}: {e}")

else:
    print("No image paths found in input_img_paths to test.")

<>:17: SyntaxWarning: invalid escape sequence '\.'
<>:20: SyntaxWarning: invalid escape sequence '\.'
<>:17: SyntaxWarning: invalid escape sequence '\.'
<>:20: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-2941883299.py:17: SyntaxWarning: invalid escape sequence '\.'
  if tf.strings.regex_full_match(tf.strings.lower(sample_image_path), ".*\.jpg$|.*\.jpeg$"):
/tmp/ipython-input-2941883299.py:20: SyntaxWarning: invalid escape sequence '\.'
  elif tf.strings.regex_full_match(tf.strings.lower(sample_image_path), ".*\.png$"):


Attempting to load and decode sample image: /content/drive/MyDrive/ObjectDetection-Tasks/Task-for-model-2/retrainDataset/combinedRetrainData/imageData/Salinan frame_20250920_114137_130922.jpg
Successfully read raw image file: /content/drive/MyDrive/ObjectDetection-Tasks/Task-for-model-2/retrainDataset/combinedRetrainData/imageData/Salinan frame_20250920_114137_130922.jpg
Attempting to decode as JPEG: /content/drive/MyDrive/ObjectDetection-Tasks/Task-for-model-2/retrainDataset/combinedRetrainData/imageData/Salinan frame_20250920_114137_130922.jpg
Successfully decoded sample image with shape: (1080, 1920, 3)


In [55]:
# Train the model
history = model_unet.fit(
    train_dataset, # Use the TensorFlow dataset for training
    epochs=100,
    validation_data=val_dataset, # Use the TensorFlow dataset for validation
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=5)  # Reduces learning rate when validation loss plateaus
    ]
)

Epoch 1/100


ValueError: Creating variables on a non-first call to a function decorated with tf.function.

In [ ]:
# # Concept based on ProgLearn tutorial :cite[9]
# # Note: This is a structural outline; exact class names and APIs may need verification from the latest ProgLearn documentation.

# # 1. Import necessary libraries
# import proglearn as PL
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input
# #from Unet import unet  # Custom function from the tutorial
# #from functions.scene_segmentation_nn_tutorial_functions import unet  # Custom function from the tutorial
# from sklearn.model_selection import train_test_split

# # 2. Load and preprocess your data (X: images, y: labels)
# # ... (data loading code as shown in the tutorial)
# #X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# # 3. Define your base neural network model (e.g., U-Net)
# #input_layer = Input(shape=train_images[0].shape)
# base_model = model_unet(n_classes=5)  # n_classes from the tutorial example

# # 4. Create the ProgLearn lifelong learner
# # The LifelongClassificationNetwork is intended to wrap a neural network.
# lifelong_nn = PL.LifelongClassificationNetwork(
#     base_model=base_model,  # Your compiled Keras model
#     # ... other parameters specific to the lifelong learner
# )

# # 5. Train the pipeline using the lifelong learning interface
# # The .fit method should handle the sequential learning.
# lifelong_nn.fit(train_images, train_labels)

# # 6. Make predictions and evaluate
# predictions = lifelong_nn.predict(test_images)
# # ... evaluation code